# Lingo Algorithm
- Create TF-IDF Matrix
- SVD TF-IDF: choose k via elbow in ROC of retained variance 
- Find cluster labels via strongest relationships between words and concepts in reduced V matrix 
- Assign documents to clusters based on document-label relationship strength
- Combine clusters with overlapping labels
- Unstem label
- Sort clusters
- Calculate statistics for comparison

In [1]:
import nltk
from nltk.corpus import reuters
import re
import numpy as np
import pandas as pd
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import sklearn
from scipy.cluster.hierarchy import dendrogram, linkage, ward, fcluster
import networkx as nx
import collections
import math
import operator
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.decomposition import PCA
from kneed import KneeLocator
from sklearn.manifold import TSNE
from nltk.stem.porter import PorterStemmer
import pickle
import gc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gimli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Tf-Idf Matrix
- Stemming 
- Tokenize
- Stop word removal

In [2]:
def stemming_tokenizer(str_input):
    
    # initialize Porter Stemmer
    stemmer = PorterStemmer()

    # tokenize input words and stem
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [stemmer.stem(word) for word in words]
    
    return words

In [3]:
def tf_idf(df):
    
    stemmer = PorterStemmer()
    
    # stem stop words before removing
    stop_stem = [stemming_tokenizer(t) for t in stopwords.words('english')]
    stop_stem = [item for sublist in stop_stem for item in sublist]
    
    # tfidf: stop word removal, tokenize and stem
    # set max number of features else memory issues
    tfidf = TfidfVectorizer(stop_words = stop_stem, tokenizer = stemming_tokenizer, max_features = 5000)
    # fit tfidf to corpus
    m = tfidf.fit_transform(df['text'])
    
    # record words in corpus with index
    feature_names = tfidf.get_feature_names() 

    return m, feature_names

### Remove Search from TF-IDF Matrix  
Else will often present the search query as a good label for a cluster. Also do not want to include as a clustering factor. 

In [4]:
def remove_search(tfidf, feature_names, search):
    try: # sometimes search already removed (ex stop word)
        # find index of search in feature_names
        search_index = feature_names.index(search)
        
        # delete from feature_names and tfidf matrix 
        cols = list(range(0,len(feature_names)))
        del cols[cols.index(search_index)]
        tfidf = tfidf[:,cols]
        del feature_names[search_index]
        
    # if error, should be a value error because search not in feature_names
    except ValueError: 
        pass
    except:
        raise 'unknown error'
    
    return tfidf, feature_names

### SVD 
SVD decomposition of tf-idf matrix

In [5]:
def svd_calculate(tfidf):
    U, S, Vt = np.linalg.svd(tfidf.todense(), full_matrices = False)
    V = Vt.T
    return U, S, V

### Find K: Number of Factors to Keep in SVD
Calculate retained varaiance at each possible value of k   
Calculate rate of change  of retained varaince    
    
k is also the number of clusters

In [6]:
def roc_var_calculate(S): # input is S matrix from SVD (singular values)

    # calculate retained variance for values of k (sum up to that value of k)
    k_var_lst = []
    k_var = 0
    for i in S:
        k_var += i**2
        k_var_lst.append(k_var)
        
    # rate of change of variance
    roc = []
    for k in range(len(k_var_lst)):
        if k+1 < len(k_var_lst):
            roc.append(abs(k_var_lst[k+1] - k_var_lst[k]) / k_var_lst[k])
    
    return roc

Find elbow in variance ROC curve 

In [7]:
def find_knee(roc):
    # knee in variance ROC 
    kn = KneeLocator(range(len(roc)), roc, curve='convex', direction='decreasing')
    if kn.knee == None: # sometimes there is no knee, just take the max k in that case
        return len(roc)
    k = kn.knee + 1 # index starts at 0 -- if knee = 1, then is roc spot 2. Between k = 2 and k = 3 --> k = 2 
    
    # need at least 2 clusters. Sometimes returns knee such k = 1 
    if k < 2:
        return 2
    
    return k 

### Reduce Dimensionality of V matrix based on K

In [8]:
def reduce_V(V, k):
    # zero out non-selected k's
    V = V[:,:k]     
    return V 

### Find Cluster Labels: Top 3 words in each concept (V column)
Record label-concept score for sorting clusters   
Finding labels that "describe" each concept bests: has highest label/word-concept score

In [9]:
def find_labels(V, feature_names): 
    # need to record labels in two different formats: 
        # dictionary where key is 1, 2, or 3: best words for each concept, second best words etc.
            # used to assign documents to clusters 
        # dictionary where key is the concept and values are the corresponding labels
            # used for reporting cluster labels
            # also record label-concept score to sort final clusters and to find labels of combined clusters in a later step
    
    # 1. key = top 1, 2, 3
    max_w3 = dict()
    max_w3_score = dict()
    # find 3 words: loop 3 times
    for i in range(1,4):
        max_w3[i] = []
        max_w3_score[i] = []
        # loop through columns of V (rows of V.T) and find ith strongest word (largest word-concept strength)
        for r in range(len(V.T)):
            max_w3[i].append(np.array(V.T[r])[0].argsort()[-i:][::-1][i-1])
    
    # 2. key = concept
    max_w_concept = dict()
    max_w_concept_score = dict()
    max_score = dict()
    # loop through columns of V (rows of V.T)
    for r in range(len(V.T)):
        # find top 3 scored words in each concept/cluster 
        if r in max_w_concept:
            max_w_concept[r].append(np.array(V.T[r])[0].argsort()[-3:][::-1])
            # record word-concept score for each word in label 
            max_w_concept_score[r].append((np.sort(np.array(V.T[r])[0])[-3:][::-1]))
        else:
            max_w_concept[r] = [np.array(V.T[r])[0].argsort()[-3:][::-1]]
            # record word-concept score for each word in label 
            max_w_concept_score[r] = [np.sort(np.array(V.T[r])[0])[-3:][::-1]]
            
        # overall max score for the concept: used to sort final clusters
        max_score[r] = np.max(max_w_concept_score[r])

    # find corresponding words corresponding to the index numbers recorded in max_w_concept
    # these are the labels
    labels = dict()
    # loop through each concept/cluster
    for k,v in max_w_concept.items():
        # record corresponding words in feature_names as labels
        labels[k] = []
        for w in v[0]:
            labels[k].append(feature_names[w])
            
    return labels, max_w3, max_w_concept, max_w_concept_score, max_score

### Assign Documents to Clusters: Label-Document Score
If document-label score above a threshold for _any_ of the three words in the label for each cluster

In [10]:
def find_docs(feature_names, max_w, m):
    # term-concept label matrix 
    # term-term matrix is identity because currently no phrases. 

    # tf-idf vectors for each of the 1st, 2nd, and 3rd words in each cluster 
    Q1 = np.identity(len(feature_names))[:,max_w[1]]
    Q2 = np.identity(len(feature_names))[:,max_w[2]]
    Q3 = np.identity(len(feature_names))[:,max_w[3]]

    # find label-document strength: tf-idf vectors for each label word by tf-idf matricies for documents
        # cij = strength of membership of jth document to ith concept 
    C1 = np.matmul(Q1.T, m.T.toarray())
    C2 = np.matmul(Q2.T, m.T.toarray())
    C3 = np.matmul(Q3.T, m.T.toarray())
    
    # choose documents for each cluster with strength > 0.1 
    # any of the top 3 words in the cluster 
    docs = dict()
    for r in range(len(C1)):
        docs[r] = []
        for c in range(len(C1[r])): 
            if C1[r][c] > 0.1 or C2[r][c] > 0.1 or C3[r][c] > 0.1:
                docs[r].append(c)

    # documents can be in multiple clusters 
    # documents can be in no clusters 

    return docs

### Combine Clusters with Overlapping Labels    
If any of the 3 label words are the same:
- Combine documents into 1 cluster
- Choose the 3 highest ranking labels between the two clusters 

In [11]:
# helper function: combine dictionary d (labels, label scores) based on combined clusters
def combine_cluster_dictionaries(d, combo_clusters):
    del_keys = []
    # for each cluster to be combined, add values to master cluster dictionary and drop absorbed cluster 
    for k,v in combo_clusters.items():
        if len(combo_clusters[k]) > 0:
            for i in v:
                d[k] += d[i]
                del_keys.append(i)
    # delete absorbed clusters
    for i in del_keys:
        del d[i]
        
    return d

In [12]:
def combine_clusters(labels, max_w_concept_score, max_w_concept,  max_score, docs, feature_names):
    
    # find clusters that contain any of the same label words
    combo_clusters = dict() # dictionary of cluster# : [clusters to be combined with key cluster]
    used_clusters = []
    # loop through labels. For each label, check all following labels for overlap.
    for k in labels.keys():
        combo_clusters[k] = list()
        for k2 in labels.keys():
            # check cluster hasn't already been marked for combination
            if k < k2 and k2 not in used_clusters: 
                # if overlapping label words, record k and k2 are clusters to be combined
                intersect = set(labels[k]).intersection(set(labels[k2]))
                if len(intersect) != 0 and k2 not in used_clusters: 
                    combo_clusters[k].append(k2)
                    used_clusters.append(k2)
    
    # if no clusters need to be combined, return original labels etc.
    if len(combo_clusters) == 0:
        return docs, labels, max_w_concept_score, max_w_concept
    
    # for dictionaries with values of scores and label indices, combine clusters and delete absorbed clusters
    max_w3_score = combine_cluster_dictionaries(max_w_concept_score, combo_clusters)
    max_w3 = combine_cluster_dictionaries(max_w_concept, combo_clusters)
    max_score = combine_cluster_dictionaries(max_score, combo_clusters)
    
    # flatten lists
    for k in max_w_concept_score.keys():
        max_w_concept_score[k] = [item for sublist in max_w_concept_score[k] for item in sublist]
    for k in max_w_concept.keys():
        max_w_concept[k] = [item for sublist in max_w_concept[k] for item in sublist]
        
        
    # generate labels of combined clusters: dictionaries of label indicies now longer than 3 after combination 
    # indices in max_w_concept with the max 3 scores in combined max_w_concept_score per cluster
    combined_labels = dict()
    count = 0
    # loop through label lists
    for k in max_w_concept_score.keys():
        # take words in label list with the highest concept-label scores (max_w_concept_score)
        # take top 5 because there could be duplicates from combination: narrow to 3 later
        label_keys = np.array(max_w_concept[k])[list(np.array(max_w_concept_score[k]).argsort()[-5:][::-1])] 
        # find indicies in feature_arrays, unique values only -> drop duplicates
        # sorted such that most important label (highest score) is first (np.unique naturally sorts)
        indexes = list(np.unique(np.array(feature_names)[label_keys], return_index = True)[1])
        # record corresponding words to indexes in sorted importance order as new labels of combined clusters
        combined_labels[count] = [np.array(feature_names)[label_keys][index] for index in sorted(indexes)]        
        count += 1
        
    # limit labels to first 3 words (most important)
    for k,v in combined_labels.items():
        combined_labels[k] = v[:3]

        
    # combine documents based on combined clusters 
    docs = combine_cluster_dictionaries(docs, combo_clusters)
    
    # reset keys to logical values (numerically ascending)
    count = 0
    docs_copy = docs.copy()
    for k in docs_copy.keys():
        docs[count] = docs.pop(k)
        count += 1

    count = 0
    max_score_copy = max_score.copy()
    for k in max_score_copy.keys():
        max_score[count] = max_score.pop(k)
        count += 1
        
    return docs, combined_labels, max_score

### Un-Stem Labels
Find the most frequent word corresponding to each stem out of the documents returned for a search   
Replace stems in labels with that word    
Else stems are not always human readable

In [13]:
def labels_unstem_create(df, labels, search):
    
    stemmer = PorterStemmer()

    # generate fresh tfidf without stemming
    tfidf = TfidfVectorizer(stop_words = stopwords.words('english'))
    tfidf_norm = tfidf.fit_transform(df['text'])
    words = tfidf.get_feature_names()

    # record stemmed versions of all words in a df
    aux = pd.DataFrame(words, columns =['word'] )
    aux['word_stemmed'] = aux['word'].apply(lambda x : stemmer.stem(x))
    
    # count the frequency of all words 
    vec = sklearn.feature_extraction.text.CountVectorizer().fit(df['text'])
    bag_of_words = vec.transform(df['text'])
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = pd.DataFrame(words_freq)
    words_freq.columns = ['word', 'num']
    
    # merge frequency of words with stemming. 
    # sort such that first instance of a stem corresponds to the most frequent variation of that stemmed word
    aux = pd.merge(aux, words_freq, on = 'word', how = 'left')
    aux = aux.sort_values(['word_stemmed', 'num'], ascending = False)
            
    # remove search term from auxilliary if it exists
    try:
        aux = aux[aux.word != search]
    except: 
        pass
    
    # loop through labels and grab the most frequent un-stemmed word for each stem encountered
    labels_unstem = collections.OrderedDict()
    # loop through labels
    for i in labels.keys():
        labels_unstem[i] = []
        # loop through words in a each label
        for j in labels[i]:
            if len(aux[aux.word_stemmed == j]) == 0:
                labels_unstem[i].append(j)
                continue
            # else take most frequent unstemmed word
            labels_unstem[i].append(aux[aux.word_stemmed == j].word.values[0])
                
    return labels_unstem

## Assign Clusters in DF 
Add list of assigned clusters to DF with document IDs

In [14]:
def cluster_df(df, docs):
    # create dataframe that indicates which documents belong to which cluster and labels. List of clusters. 
    framesvd = df
    framesvd['cluster'] = ''
    for k,v in docs.items():
        for d in v: 
            framesvd.cluster = np.where(framesvd.index == d, framesvd.cluster + str(k) + ',', framesvd.cluster)

    # create listof clusters
    framesvd.cluster = framesvd.cluster.str.split(',')
    # drop blanks
    framesvd.cluster = framesvd.cluster.apply(lambda row: [i for i in row if i != ''])
    # drop duplicates
    framesvd.cluster = framesvd.cluster.apply(set)
    framesvd.cluster = framesvd.cluster.apply(list)
    # convert into integers
    framesvd.cluster = framesvd.cluster.apply(lambda row: [int(i) for i in row])
    
    return framesvd

## Sort Clusters
Max label-concept score * size of cluster

In [15]:
def label_sort(labels, max_score, frame):
    # multiply max_score by number of documents in that cluster: prefer large and well separated clusters 
    for k,v in max_score.items():
        max_score[k] = max_score[k] * len(frame[frame.cluster.map(set([k]).issubset)])
    
    # sort max score
    max_score = sorted(max_score.items(), key=operator.itemgetter(1), reverse = True)
    
    # add values in sorted order to labels: ordered dictionary
    labels_sorted = collections.OrderedDict()
    for i in max_score:
        labels_sorted[i[0]] = labels[i[0]]
        
    return labels_sorted

# Calculate Metrics
Not used to calculate clusters - just for evaluation and comparison with Hierarchical

__Euclidean Distance: TFIDF__

In [16]:
def dist_calculate(tfidf):
    dist = euclidean_distances(tfidf)  
    return dist

__Find Cluster Centroids__    
Mean TFIDF vector among documents in each cluster

In [17]:
def find_centroids(frameh, tfidf, k):
    # most common words in clusters (based on tf-idf not just frequency)
    centroid = dict()
    # loop through clusters
    for c in range(k):
        # identify documents in cluster 
        cluster1 = list(frameh[frameh.cluster.map(set([c]).issubset)].index.unique())
        # mean vector of tf-idf vectors of documents in cluster
        tfidf1 = tfidf[cluster1,:]
        tfidf1 = tfidf1.mean(axis = 0)
        centroid[c] = tfidf1
        
    return centroid

__Calculate Silhouette Score__  
Return overall average

In [18]:
# calculate individual silhouette for each point 
def silhouette_individ(frameh, dist, k):
    
    # average distance to points in own cluster
    sil_a = dict()
    # loop through clusters
    for c in range(k):
        sil_a[c] = dict()
        # all documents in cluster c 
        docs_i = list(frameh[frameh.cluster.map(set([c]).issubset)].index.unique())
        # if one document in cluster, then avg distance = 0
        if len(docs_i) == 1:
            sil_a[c][docs_i[0]] = 0
        else:
            # loop through points in each cluster and take mean distance from other points in documents
            for i in docs_i:
                docs_i.remove(i)
                sil_a[c][i] = np.nanmean(dist[i, docs_i].tolist())
                
    # minimum average distance to points in other clusters 
    # for each point, create list of average distances to points in each other cluster. Take minimum of list
    sil_b = dict()
    # loop through clusters
    for c in range(k):
        sil_b[c] = dict()
        # all documents in cluster c
        docs_in = list(frameh[frameh.cluster.map(set([c]).issubset)].index.unique())
        # loop through points in each cluster -> loop through other clusters -> loop through documents in that cluster 
        for i in docs_in:
            lst = []
            for c2 in range(k):
                if c2 != c:
                    docs_out = list(frameh[frameh.cluster.map(set([c2]).issubset)].index.unique())
                    if i in docs_out: # if target document is in cluster (can be in multiple), remove 
                        docs_out.remove(i)
                    # loop through documents in other cluster and find average distance
                    lst.append(np.nanmean(dist[i,docs_out].tolist()))
                
            # take minimum of average distance to other clusters
            sil_b[c][i] = np.min(lst)
            
    return sil_a, sil_b

In [19]:
def silhouette_avg_calculate(frameh, dist, k):
    # silhouette components from each individual point
    sil_a, sil_b = silhouette_individ(frameh, dist, k)
    
    # take average among points in same cluster to get cluster-level silhouette
    sil_scores = dict()
    for k,v in sil_a.items():
        lst = []
        for i in range(len(v.values())):
            max_ab = max(list(sil_b[k].values())[i], list(sil_a[k].values())[i])
            min_ab = min(list(sil_b[k].values())[i], list(sil_a[k].values())[i])
            # calculate silhouette score 
            lst.append(1 - min_ab/max_ab)
        sil_scores[k] = np.nanmean(lst) 
       
    # take average among all clusters to get overall silhouette
    return np.nanmean(list(sil_scores.values()))

__Calculate Distortion__   
Calculate total and average distortion of a clustering 

In [20]:
def distortion_calculate(tfidf, centroid, frameh):
    sumd = 0
    countpts = 0
    # sum together distortion for each cluster 
    for i in list(frameh.index.unique()):
        for c in frameh[frameh.index == i].cluster.tolist()[0]:
            sumd += np.linalg.norm(tfidf[i]-centroid[c])**2
            countpts +=1 
        
    return sumd, sumd/countpts

# Main Function

In [21]:
def main():
    
    # read in processed data
    df = pd.read_pickle('reuters_processed')
    
    # set of topics/search terms
    topics = list(df.categories)
    topics = [item for sublist in topics for item in sublist]
    topics = list(set(topics))

    # initialize data structures to store stats for each search query/topic's clustering
    df_final = pd.DataFrame()
    labels_dict = dict()
    k_dict = dict()
    silhouette_dict = dict()
    distortion_dict = dict()
    max_score_dict = dict()
    percent_zero_dict = dict()
    percent_mult_dict = dict()
    cluster1 = []

    # loop through all search queries 
    for search in topics:
        gc.collect()
        
        # subset of dataframe for search
        df_subset = df[df.categories.map(set([search]).issubset)] 
        df_subset = df_subset.reset_index()
        
        # if less than 5 documents in search, enforce 1 cluster
        if len(df_subset) < 5:
            cluster1.append(search)
            continue
            
        # skip two search terms that return a large number of serach tera
        if search == 'earn' or search == 'acq': # too big to deal with
            continue
        
        print(search)
        
        # TF-IDF matrix
        tfidf, feature_names = tf_idf(df_subset)
                
        # remove search from tf-idf matrix so is not a clustering factor or a label 
        tfidf, feature_names = remove_search(tfidf, feature_names, search)

        # SVD 
        U, S, V = svd_calculate(tfidf)

        # Find K via elbow in ROC of retained variance and reduce dimensionality
        roc = roc_var_calculate(S)
        k = find_knee(roc) 
        V = reduce_V(V, k)

        # Find cluster labels: max concept-word scores 
        labels, max_w3, max_w_concept, max_w_concept_score, max_score = find_labels(V, feature_names)
        
        # Assign documents to clusters based on labels
        docs = find_docs(feature_names, max_w3, tfidf)
                
        # delete cluster if no documents in it (no documents pass threshold)
        for k,v in docs.copy().items():
            if len(docs[k]) == 0:
                del docs[k]
                del labels[k]
                del max_score[k]
                del max_w_concept_score[k]
                del max_w_concept[k]
                
        # combine clusters with overlapping labels
        docs, labels, max_score = combine_clusters(labels, max_w_concept_score, max_w_concept, max_score, docs, feature_names)

        # mark clusters in dataframe
        frame = cluster_df(df_subset, docs)

        # sort clusters
        labels = label_sort(labels, max_score, frame)
        
        # unstem labels
        labels = labels_unstem_create(df_subset, labels, search)        
        
        # if process returned only one cluster (after merging), skip the rest of the calculations 
        if len(labels) == 1:
            cluster1.append(search)
            continue
            
        # percent of documents with no cluster
        frame['len']= frame.cluster.apply(lambda row: len(row))
        percent_zero_dict[search] = len(frame[frame.len == 0]) / len(frame)
        
        # percent of documents in multiple clusters
        percent_mult_dict[search] = len(frame[frame.len > 1]) / len(frame)
        
        # calculate metrics for comparison
        dist = dist_calculate(tfidf)
        centroid = find_centroids(frame, tfidf, len(labels))
        silhouette = silhouette_avg_calculate(frame, dist, len(labels))
        distortion, distortion_avg = distortion_calculate(tfidf, centroid, frame)
        
        # record statistics for this serach 
        labels_dict[search] = labels
        k_dict[search] = len(labels) # don't use k because combined clusters after that step
        silhouette_dict[search] = silhouette
        distortion_dict[search] = distortion_avg
        max_score_dict[search] = max_score

        frame['search'] = search
        df_final = df_final.append(frame)
        
    return df_final, labels_dict, percent_zero_dict, percent_mult_dict, k_dict, distortion_dict, silhouette_dict, cluster1

In [22]:
df_final, labels_dict, percent_zero_dict, percent_mult_dict, k_dict, distortion_dict, silhouette_dict, cluster1 = main()

soy-oil
oilseed
ship
rubber
instal-debt
retail


C:\Users\Gimli\Anaconda3\lib\site-packages\kneed\knee_locator.py:188: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


groundnut
pet-chem
soybean
coconut
housing


C:\Users\Gimli\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: Mean of empty slice


trade
heat
naphtha
grain
coconut-oil
silver
lead
corn
meal-feed
gas
oat
wpi
rice
dmk
jobs
yen
reserves
interest
fuel
coffee
propane
money-fx
lumber
carcass
orange
alum
crude
tea
sorghum
barley
income
nickel
jet
gnp
strategic-metal
money-supply
lei
wheat
bop
dlr
rape-oil
sugar
copper
ipi
sun-oil
tin
gold
soy-meal
veg-oil
rapeseed
livestock
palm-oil
l-cattle
cocoa
platinum
potato
iron-steel
cpi
hog
cotton
nat-gas
zinc
sunseed


In [23]:
# save data locally
with open('lingo', "wb") as f:
    pickle.dump(df_final, f)
    pickle.dump(labels_dict, f)
    pickle.dump(k_dict, f)
    pickle.dump(distortion_dict, f)
    pickle.dump(silhouette_dict, f)
    pickle.dump(percent_zero_dict, f)
    pickle.dump(percent_mult_dict, f)
    pickle.dump(cluster1, f)